# Introduction
- このノートブックでは、新しい特徴量(角度)の生成を試みる
- ref
    - https://www.kaggle.com/kmat2019/effective-feature
    - https://www.kaggle.com/kernels/scriptcontent/15983586/download

# Import everything I need :)

In [1]:
import time
import glob
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
# from fastprogress import progress_bar

/usr/local/lib/python3.6/dist-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


# Preparation

In [2]:
nb = 20
isSmallSet = True
length = 10000

In [3]:
pd.set_option('display.max_columns', 100)

In [4]:
file_path = '../input/champs-scalar-coupling/'
glob.glob(file_path + '*')

['../input/champs-scalar-coupling/magnetic_shielding_tensors.csv',
 '../input/champs-scalar-coupling/structures.csv',
 '../input/champs-scalar-coupling/mulliken_charges.csv',
 '../input/champs-scalar-coupling/structures.zip',
 '../input/champs-scalar-coupling/scalar_coupling_contributions.csv',
 '../input/champs-scalar-coupling/train.csv',
 '../input/champs-scalar-coupling/test.csv',
 '../input/champs-scalar-coupling/sample_submission.csv',
 '../input/champs-scalar-coupling/dipole_moments.csv',
 '../input/champs-scalar-coupling/potential_energy.csv']

In [5]:
# train
path = file_path + 'train.csv'
if isSmallSet:
    train = pd.read_csv(path) [:length]
else:
    train = pd.read_csv(path)

In [6]:
# test
path = file_path + 'test.csv'
if isSmallSet:
    test = pd.read_csv(path)[:length]
else:
    test = pd.read_csv(path)

In [7]:
# structure
path = file_path + 'structures.csv'
structures = pd.read_csv(path)

In [8]:
if isSmallSet:
    print('using SmallSet !!')
    print('-------------------')

print(f'There are {train.shape[0]} rows in train data.')
print(f'There are {test.shape[0]} rows in test data.')

print(f"There are {train['molecule_name'].nunique()} distinct molecules in train data.")
print(f"There are {test['molecule_name'].nunique()} distinct molecules in test data.")
print(f"There are {train['atom_index_0'].nunique()} unique atoms.")
print(f"There are {train['type'].nunique()} unique types.")

using SmallSet !!
-------------------
There are 10000 rows in train data.
There are 10000 rows in test data.
There are 312 distinct molecules in train data.
There are 266 distinct molecules in test data.
There are 19 unique atoms.
There are 8 unique types.


## myFunc

In [9]:
import pandas as pd
import numpy as np
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

# Feature Engineering

**basic**

In [10]:
def map_atom_info(df_1,df_2, atom_idx):
    df = pd.merge(df_1, df_2, how = 'left',
                  left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                  right_on = ['molecule_name',  'atom_index'])
    df = df.drop('atom_index', axis=1)

    return df


for atom_idx in [0,1]:
    train = map_atom_info(train, structures, atom_idx)
    test  = map_atom_info(test, structures, atom_idx)
    
    train = train.rename(columns={'atom': f'atom_{atom_idx}',
                                        'x': f'x_{atom_idx}',
                                        'y': f'y_{atom_idx}',
                                        'z': f'z_{atom_idx}'})
    test  =  test.rename(columns={'atom': f'atom_{atom_idx}',
                                        'x': f'x_{atom_idx}',
                                        'y': f'y_{atom_idx}',
                                        'z': f'z_{atom_idx}'})
# train = map_atom_info(train, structures, 0)
# train = map_atom_info(train, structures, 1)

# test = map_atom_info(test, structures, 0)
# test = map_atom_info(test, structures, 1)

distances

In [11]:
train_p_0 = train[['x_0', 'y_0', 'z_0']].values
train_p_1 = train[['x_1', 'y_1', 'z_1']].values
test_p_0 = test[['x_0', 'y_0', 'z_0']].values
test_p_1 = test[['x_1', 'y_1', 'z_1']].values

train['dist'] = np.linalg.norm(train_p_0 - train_p_1, axis=1)
test['dist'] = np.linalg.norm(test_p_0 - test_p_1, axis=1)
# train['dist_x'] = (train['x_0'] - train['x_1']) ** 2
# test['dist_x'] = (test['x_0'] - test['x_1']) ** 2
# train['dist_y'] = (train['y_0'] - train['y_1']) ** 2
# test['dist_y'] = (test['y_0'] - test['y_1']) ** 2
# train['dist_z'] = (train['z_0'] - train['z_1']) ** 2
# test['dist_z'] = (test['z_0'] - test['z_1']) ** 2

In [12]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,dist
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,1.091953
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,1.783120
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,1.783147
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,1.783157
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,1.091952


---

**以下、con feature を作る**

あるatomと相互作用している最も距離の近いatomを探す。

まずatom_index_0とatom_index1に分けて情報がはいっているため、一元化する。

In [12]:
#I apologize for my poor coding skill. Please make the better one.
print(train.shape)
df_temp=train.loc[:,["molecule_name","atom_index_0","atom_index_1","dist","x_0","y_0","z_0","x_1","y_1","z_1"]].copy()

df_temp_=df_temp.copy()
df_temp_= df_temp_.rename(columns={'atom_index_0': 'atom_index_1',
                                   'atom_index_1': 'atom_index_0',
                                   'x_0': 'x_1',
                                   'y_0': 'y_1',
                                   'z_0': 'z_1',
                                   'x_1': 'x_0',
                                   'y_1': 'y_0',
                                   'z_1': 'z_0'})

display(df_temp.head(1))
display(df_temp_.head(1))

(10000, 15)


,molecule_name,atom_index_0,atom_index_1,dist,x_0,y_0,z_0,x_1,y_1,z_1
0,dsgdb9nsd_000001,1,0,1.091953,0.00215,-0.006031,0.001976,-0.012698,1.085804,0.008001


,molecule_name,atom_index_1,atom_index_0,dist,x_1,y_1,z_1,x_0,y_0,z_0
0,dsgdb9nsd_000001,1,0,1.091953,0.00215,-0.006031,0.001976,-0.012698,1.085804,0.008001


In [13]:
df_temp=pd.concat((df_temp,df_temp_),axis=0)
print(len(df_temp))
df_temp.head(1)

20000


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




,atom_index_0,atom_index_1,dist,molecule_name,x_0,x_1,y_0,y_1,z_0,z_1
0,1,0,1.091953,dsgdb9nsd_000001,0.00215,-0.012698,-0.006031,1.085804,0.001976,0.008001


---> 一元化完了

次は、最小距離を探す

In [14]:
df_temp["min_dist"]=df_temp.groupby(['molecule_name', 'atom_index_0'])['dist'].transform('min')
df_temp= df_temp[df_temp["min_dist"]==df_temp["dist"]]

In [15]:
df_temp=df_temp.drop(['x_0','y_0','z_0','min_dist'], axis=1)
df_temp= df_temp.rename(columns={'atom_index_0': 'atom_index',
                                 'atom_index_1': 'atom_index_closest',
                                 'dist': 'dist_closest',
                                 'x_1': 'x_closest',
                                 'y_1': 'y_closest',
                                 'z_1': 'z_closest'})

print(df_temp.duplicated(subset=['molecule_name', 'atom_index']).value_counts())
#delete duplicated rows (some atom pairs have perfectly same distance)
#This code is added based on Adriano Avelar's comment.
df_temp=df_temp.drop_duplicates(subset=['molecule_name', 'atom_index'])

for atom_idx in [0,1]:
    train = map_atom_info(train,df_temp, atom_idx)
    train = train.rename(columns={'atom_index_closest': f'atom_index_closest_{atom_idx}',
                                        'dist_closest': f'dist_closest_{atom_idx}',
                                        'x_closest': f'x_closest_{atom_idx}',
                                        'y_closest': f'y_closest_{atom_idx}',
                                        'z_closest': f'z_closest_{atom_idx}'})

print(train.shape)
train.head()

False    3360
dtype: int64
(10000, 25)


,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,dist,atom_index_closest_0,dist_closest_0,x_closest_0,y_closest_0,z_closest_0,atom_index_closest_1,dist_closest_1,x_closest_1,y_closest_1,z_closest_1
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,1.091953,0,1.091953,-0.012698,1.085804,0.008001,3,1.091946,-0.540815,1.447527,-0.876644
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,1.783120,0,1.091953,-0.012698,1.085804,0.008001,0,1.091952,-0.012698,1.085804,0.008001
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,1.783147,0,1.091953,-0.012698,1.085804,0.008001,0,1.091946,-0.012698,1.085804,0.008001
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,1.783157,0,1.091953,-0.012698,1.085804,0.008001,0,1.091948,-0.012698,1.085804,0.008001
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,1.091952,0,1.091952,-0.012698,1.085804,0.008001,3,1.091946,-0.540815,1.447527,-0.876644


---

**angle**

In [16]:
def add_cos_features(df):
    df["dist_0"]=((df['x_0']-df['x_closest_0'])**2+(df['y_0']-df['y_closest_0'])**2+(df['z_0']-df['z_closest_0'])**2)**(1/2)
    df["dist_1"]=((df['x_1']-df['x_closest_1'])**2+(df['y_1']-df['y_closest_1'])**2+(df['z_1']-df['z_closest_1'])**2)**(1/2)
    df["vec_0_x"]=(df['x_0']-df['x_closest_0'])/df["dist_0"]
    df["vec_0_y"]=(df['y_0']-df['y_closest_0'])/df["dist_0"]
    df["vec_0_z"]=(df['z_0']-df['z_closest_0'])/df["dist_0"]
    df["vec_1_x"]=(df['x_1']-df['x_closest_1'])/df["dist_1"]
    df["vec_1_y"]=(df['y_1']-df['y_closest_1'])/df["dist_1"]
    df["vec_1_z"]=(df['z_1']-df['z_closest_1'])/df["dist_1"]
    df["vec_x"]=(df['x_1']-df['x_0'])/df["dist"]
    df["vec_y"]=(df['y_1']-df['y_0'])/df["dist"]
    df["vec_z"]=(df['z_1']-df['z_0'])/df["dist"]
    df["cos_0_1"]=df["vec_0_x"]*df["vec_1_x"]+df["vec_0_y"]*df["vec_1_y"]+df["vec_0_z"]*df["vec_1_z"]
    df["cos_0"]=df["vec_0_x"]*df["vec_x"]+df["vec_0_y"]*df["vec_y"]+df["vec_0_z"]*df["vec_z"]
    df["cos_1"]=df["vec_1_x"]*df["vec_x"]+df["vec_1_y"]*df["vec_y"]+df["vec_1_z"]*df["vec_z"]
    df=df.drop(['vec_0_x','vec_0_y','vec_0_z','vec_1_x','vec_1_y','vec_1_z','vec_x','vec_y','vec_z'], axis=1)
    return df
    
train = add_cos_features(train)

In [17]:
train.head(1)

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,dist,atom_index_closest_0,dist_closest_0,x_closest_0,y_closest_0,z_closest_0,atom_index_closest_1,dist_closest_1,x_closest_1,y_closest_1,z_closest_1,dist_0,dist_1,cos_0_1,cos_0,cos_1
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.00215,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,1.091953,0,1.091953,-0.012698,1.085804,0.008001,3,1.091946,-0.540815,1.447527,-0.876644,1.091953,1.091946,0.333335,-1.0,-0.333335


# Summary
- add features below
    - 'atom_index_closest_0', 'dist_closest_0',
       'x_closest_0', 'y_closest_0', 'z_closest_0', 'atom_index_closest_1',
       'dist_closest_1', 'x_closest_1', 'y_closest_1', 'z_closest_1', 'dist_0',
       'dist_1', 'cos_0_1', 'cos_0', 'cos_1'

In [13]:
def add_cos_features(df):
    #I apologize for my poor coding skill. Please make the better one.
#     print(train.shape)
    df_temp=df.loc[:,["molecule_name","atom_index_0","atom_index_1","dist","x_0","y_0","z_0","x_1","y_1","z_1"]].copy()

    df_temp_=df_temp.copy()
    df_temp_= df_temp_.rename(columns={'atom_index_0': 'atom_index_1',
                                       'atom_index_1': 'atom_index_0',
                                       'x_0': 'x_1',
                                       'y_0': 'y_1',
                                       'z_0': 'z_1',
                                       'x_1': 'x_0',
                                       'y_1': 'y_0',
                                       'z_1': 'z_0'})
    df_temp=pd.concat((df_temp,df_temp_),axis=0)
    df_temp["min_dist"]=df_temp.groupby(['molecule_name', 'atom_index_0'])['dist'].transform('min')
    df_temp= df_temp[df_temp["min_dist"]==df_temp["dist"]]
    df_temp=df_temp.drop(['x_0','y_0','z_0','min_dist'], axis=1)
    df_temp= df_temp.rename(columns={'atom_index_0': 'atom_index',
                                     'atom_index_1': 'atom_index_closest',
                                     'dist': 'dist_closest',
                                     'x_1': 'x_closest',
                                     'y_1': 'y_closest',
                                     'z_1': 'z_closest'})

    #delete duplicated rows (some atom pairs have perfectly same distance)
    #This code is added based on Adriano Avelar's comment.
    df_temp=df_temp.drop_duplicates(subset=['molecule_name', 'atom_index'])

    for atom_idx in [0,1]:
        df = map_atom_info(df,df_temp, atom_idx)
        df = df.rename(columns={'atom_index_closest': f'atom_index_closest_{atom_idx}',
                                            'dist_closest': f'dist_closest_{atom_idx}',
                                            'x_closest': f'x_closest_{atom_idx}',
                                            'y_closest': f'y_closest_{atom_idx}',
                                            'z_closest': f'z_closest_{atom_idx}'})

    df["dist_0"]=((df['x_0']-df['x_closest_0'])**2+(df['y_0']-df['y_closest_0'])**2+(df['z_0']-df['z_closest_0'])**2)**(1/2)
    df["dist_1"]=((df['x_1']-df['x_closest_1'])**2+(df['y_1']-df['y_closest_1'])**2+(df['z_1']-df['z_closest_1'])**2)**(1/2)
    df["vec_0_x"]=(df['x_0']-df['x_closest_0'])/df["dist_0"]
    df["vec_0_y"]=(df['y_0']-df['y_closest_0'])/df["dist_0"]
    df["vec_0_z"]=(df['z_0']-df['z_closest_0'])/df["dist_0"]
    df["vec_1_x"]=(df['x_1']-df['x_closest_1'])/df["dist_1"]
    df["vec_1_y"]=(df['y_1']-df['y_closest_1'])/df["dist_1"]
    df["vec_1_z"]=(df['z_1']-df['z_closest_1'])/df["dist_1"]
    df["vec_x"]=(df['x_1']-df['x_0'])/df["dist"]
    df["vec_y"]=(df['y_1']-df['y_0'])/df["dist"]
    df["vec_z"]=(df['z_1']-df['z_0'])/df["dist"]
    df["cos_0_1"]=df["vec_0_x"]*df["vec_1_x"]+df["vec_0_y"]*df["vec_1_y"]+df["vec_0_z"]*df["vec_1_z"]
    df["cos_0"]=df["vec_0_x"]*df["vec_x"]+df["vec_0_y"]*df["vec_y"]+df["vec_0_z"]*df["vec_z"]
    df["cos_1"]=df["vec_1_x"]*df["vec_x"]+df["vec_1_y"]*df["vec_y"]+df["vec_1_z"]*df["vec_z"]
    df=df.drop(['vec_0_x','vec_0_y','vec_0_z','vec_1_x','vec_1_y','vec_1_z','vec_x','vec_y','vec_z'], axis=1)
    
    df = reduce_mem_usage(df)
    
    return df

In [14]:
train = add_cos_features(train)
test  = add_cos_features(test)

Mem. usage decreased to  0.84 Mb (64.5% reduction)
Mem. usage decreased to  0.84 Mb (63.3% reduction)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




In [16]:
train.head(100)

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,dist,atom_index_closest_0,dist_closest_0,x_closest_0,y_closest_0,z_closest_0,atom_index_closest_1,dist_closest_1,x_closest_1,y_closest_1,z_closest_1,dist_0,dist_1,cos_0_1,cos_0,cos_1
0,0,dsgdb9nsd_000001,1,0,1JHC,84.812500,H,0.002150,-0.006031,0.001976,C,-0.012695,1.085938,0.008003,1.091797,0,1.091797,-0.012695,1.085938,0.008003,3,1.091797,-0.541016,1.447266,-0.876465,1.091797,1.091797,0.333252,-1.000000,-0.333252
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.257812,H,0.002150,-0.006031,0.001976,H,1.011719,1.463867,0.000277,1.783203,0,1.091797,-0.012695,1.085938,0.008003,0,1.091797,-0.012695,1.085938,0.008003,1.091797,1.091797,-0.333252,-0.816406,0.816406
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.257812,H,0.002150,-0.006031,0.001976,H,-0.541016,1.447266,-0.876465,1.783203,0,1.091797,-0.012695,1.085938,0.008003,0,1.091797,-0.012695,1.085938,0.008003,1.091797,1.091797,-0.333252,-0.816406,0.816406
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.257812,H,0.002150,-0.006031,0.001976,H,-0.523926,1.437500,0.906250,1.783203,0,1.091797,-0.012695,1.085938,0.008003,0,1.091797,-0.012695,1.085938,0.008003,1.091797,1.091797,-0.333252,-0.816406,0.816406
4,4,dsgdb9nsd_000001,2,0,1JHC,84.812500,H,1.011719,1.463867,0.000277,C,-0.012695,1.085938,0.008003,1.091797,0,1.091797,-0.012695,1.085938,0.008003,3,1.091797,-0.541016,1.447266,-0.876465,1.091797,1.091797,0.333252,-1.000000,-0.333252
5,5,dsgdb9nsd_000001,2,3,2JHH,-11.257812,H,1.011719,1.463867,0.000277,H,-0.541016,1.447266,-0.876465,1.783203,0,1.091797,-0.012695,1.085938,0.008003,0,1.091797,-0.012695,1.085938,0.008003,1.091797,1.091797,-0.333252,-0.816406,0.816406
6,6,dsgdb9nsd_000001,2,4,2JHH,-11.257812,H,1.011719,1.463867,0.000277,H,-0.523926,1.437500,0.906250,1.783203,0,1.091797,-0.012695,1.085938,0.008003,0,1.091797,-0.012695,1.085938,0.008003,1.091797,1.091797,-0.333252,-0.816406,0.816406
7,7,dsgdb9nsd_000001,3,0,1JHC,84.812500,H,-0.541016,1.447266,-0.876465,C,-0.012695,1.085938,0.008003,1.091797,0,1.091797,-0.012695,1.085938,0.008003,3,1.091797,-0.541016,1.447266,-0.876465,1.091797,1.091797,-1.000000,-1.000000,1.000000
8,8,dsgdb9nsd_000001,3,4,2JHH,-11.257812,H,-0.541016,1.447266,-0.876465,H,-0.523926,1.437500,0.906250,1.783203,0,1.091797,-0.012695,1.085938,0.008003,0,1.091797,-0.012695,1.085938,0.008003,1.091797,1.091797,-0.333252,-0.816406,0.816406
9,9,dsgdb9nsd_000001,4,0,1JHC,84.812500,H,-0.523926,1.437500,0.906250,C,-0.012695,1.085938,0.008003,1.091797,0,1.091797,-0.012695,1.085938,0.008003,3,1.091797,-0.541016,1.447266,-0.876465,1.091797,1.091797,0.333252,-1.000000,-0.333252
